In [1]:
import numpy as np
import matplotlib.pyplot as plt

import tidy3d as td
import tidy3d.web as web

import optax
from autograd import value_and_grad

In [ ]:
# um to nm
nm = 1e-3

wavelength = 850 * nm
f0 = td.C_0 / wavelength
fwidth0 = f0 / 20

# NA selection
NA = 0.9

# 
H = 430 * nm
S = 320 * nm

n_Si = 3.84  # aSi
n_SiO2 = 1.45

air = td.Medium(permittivity=1.0)
SiO2 = td.Medium(permittivity=n_SiO2**2)
Si = td.Medium(permittivity=n_Si**2)